In [ ]:
from scipy.stats import poisson
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
cd ~/code/snc

In [ ]:
from collections import defaultdict

In [ ]:
from snc.meio.gsm.utils import read_supply_chain_from_txt

In [ ]:
from snc.experiment.numerical_simulator import simulate,compute_base_stocks,truncate_and_conserve
from snc.experiment.vis_utils import plot_cascading_effect
from snc.meio.gsm.tree_gsm import GuaranteedServiceModelTree
from snc.meio.gsm.tree_gsm import verify_solution_policy,compute_replenishment_times

In [ ]:
from snc.experiment.basic_serial_network import (create_serial_stages, 
                                                 iterate_experimental_profiles, 
                                                 create_serial_line_from_lead_times)
from snc.experiment.num_sim_utils import get_new_stockout_intervals

In [ ]:
n=10000
lam = 10
sla = 0.95

In [ ]:
lead_times = [1,3,10,30,100,300,1000,3000,10000]

In [ ]:
seed = 8675310
f,ax = plt.subplots(3,1,figsize=(12,18),sharex=False)

stages = create_serial_line_from_lead_times(lead_times=[1],demand_stage_params=demand_stage_params)
policy = {stage_id:{"s":0,"si":0} for stage_id in stages}
base_stocks = compute_base_stocks(stages,policy,lam,sla=sla)
capacity_constraints = {"1":12}

stat_func = np.mean
    
np.random.seed(seed=seed)
inv_histories = simulate(stages,policy,base_stocks,capacity_constraints,lam,n,stockout_stages=None)
indep_inv_history = inv_histories["1"]
indep_sla = np.mean(inv_histories["1"] >= 0)
indep_stockout_intervals = _collect_stockout_intervals(inv_histories["1"])
indep_backorders = stat_func(sum(indep_stockout_intervals.values(),[]))
indep_stout_dur = stat_func([len(interval) for interval in indep_stockout_intervals.values()])

lead_times = list(lead_times)

for n_bufs in range(5):
    effective_slas = [indep_sla]
    backorders = [indep_backorders]
    stout_durs = [indep_stout_dur]
    for l2 in lead_times:
        l_times = [1]+[1]*n_bufs+[l2]
        stages = create_serial_line_from_lead_times(lead_times=l_times,demand_stage_params=demand_stage_params)
        policy = {stage_id:{"s":0,"si":0} for stage_id in stages}
        base_stocks = compute_base_stocks(stages,policy,lam,sla=sla)

        np.random.seed(seed=seed)
        inv_histories = simulate(stages,policy,base_stocks,capacity_constraints,lam,n,stockout_stages=None)
        effective_slas.append(np.mean(inv_histories["1"][l2:] >= 0))
        stockout_intervals = get_new_stockout_intervals(inv_histories["1"],indep_inv_history)
        
        backorders.append(stat_func(sum(stockout_intervals.values(),[])))
        stout_durs.append(stat_func([len(inter) for inter in stockout_intervals.values()]))
        
        #neg = inv_histories["1"][l2:] < 0
        #bad_days = ~(neg & indep_neg[l2:]) & neg
        #backorders.append(-np.median(inv_histories["1"][l2:][bad_days]))

    ax[0].plot([5e-1]+lead_times,effective_slas,"-.b",alpha=0.2)
    ax[0].plot([5e-1]+lead_times,effective_slas,"o",label=n_bufs)
    
    ax[1].plot([5e-1]+lead_times,backorders,"-.b",alpha=0.2)
    ax[1].plot([5e-1]+lead_times,backorders,"o",label=n_bufs)
    
    ax[2].plot([5e-1]+lead_times,stout_durs,"-.b",alpha=0.2)
    ax[2].plot([5e-1]+lead_times,stout_durs,"o",label=n_bufs)

for i in range(3):
    ax[i].set_xscale("log")
    ax[i].set_xlabel("Supply lead time (days)")
    ax[i].grid(axis="y")
    ax[i].legend(title="Number of intermediate buffers")

ax[0].set_yticks(np.arange(0.9,0.97,0.01))
ax[0].set_ylabel("Effective SLA")
ax[1].set_ylabel("Mean backorders queue (items)")
ax[2].set_ylabel("Mean stockout duration (days)")
ax[0].set_title("Effect of cascading stockouts in presence of intermediate buffer stages")
ax[1].set_yscale("log")
ax[2].set_yscale("log")

In [ ]:
seed = 8675310
f,ax = plt.subplots(3,1,figsize=(12,18),sharex=False)

stages = create_serial_line_from_lead_times(lead_times=[1],demand_stage_params=demand_stage_params)
policy = {stage_id:{"s":0,"si":0} for stage_id in stages}
base_stocks = compute_base_stocks(stages,policy,lam,sla=sla)

stat_func = np.mean
    
np.random.seed(seed=seed)
inv_histories = simulate(stages,policy,base_stocks,lam,n,stockout_stages=None)
indep_inv_history = inv_histories["1"]
indep_sla = np.mean(inv_histories["1"] >= 0)
indep_stockout_intervals = _collect_stockout_intervals(inv_histories["1"])
indep_backorders = stat_func(sum(indep_stockout_intervals.values(),[]))
indep_stout_dur = stat_func([len(interval) for interval in indep_stockout_intervals.values()])


for n_bufs in range(4,5):
    effective_slas = [indep_sla]
    backorders = [indep_backorders]
    stout_durs = [indep_stout_dur]
    effective_slas_2 = [indep_sla]
    for l2 in lead_times:
        l_times = [1]+[1]*n_bufs+[l2]
        stages = create_serial_line_from_lead_times(lead_times=l_times,demand_stage_params=demand_stage_params)
        policy = {stage_id:{"s":0,"si":0} for stage_id in stages}
        base_stocks = compute_base_stocks(stages,policy,lam,sla=sla)

        np.random.seed(seed=seed)
        inv_histories = simulate(stages,policy,base_stocks,lam,n,stockout_stages=None)
        effective_slas.append(np.mean(inv_histories["1"][l2:] >= 0))
        stockout_intervals = get_new_stockout_intervals(inv_histories["1"],indep_inv_history)
        
        backorders.append(stat_func(sum(stockout_intervals.values(),[])))
        stout_durs.append(stat_func([len(inter) for inter in stockout_intervals.values()]))
        
        l_times = [1]+[n_bufs]+[l2]
        stages = create_serial_line_from_lead_times(lead_times=l_times,demand_stage_params=demand_stage_params)
        policy = {stage_id:{"s":0,"si":0} for stage_id in stages}
        base_stocks = compute_base_stocks(stages,policy,lam,sla=sla)

        np.random.seed(seed=seed)
        inv_histories = simulate(stages,policy,base_stocks,lam,n,stockout_stages=None)
        effective_slas_2 .append(np.mean(inv_histories["1"][l2:] >= 0))
        continue
        stockout_intervals = get_new_stockout_intervals(inv_histories["1"],indep_inv_history)
        
        backorders.append(stat_func(sum(stockout_intervals.values(),[])))
        stout_durs.append(stat_func([len(inter) for inter in stockout_intervals.values()]))
        
        #neg = inv_histories["1"][l2:] < 0
        #bad_days = ~(neg & indep_neg[l2:]) & neg
        #backorders.append(-np.median(inv_histories["1"][l2:][bad_days]))

    ax[0].plot([5e-1]+lead_times,effective_slas,"-.b",alpha=0.2)
    ax[0].plot([5e-1]+lead_times,effective_slas,"o",label=n_bufs)
    ax[0].plot([5e-1]+lead_times,effective_slas_2,"-.b",alpha=0.2)
    ax[0].plot([5e-1]+lead_times,effective_slas_2,"o",label="aggregated {}".format(n_bufs))
    
    ax[1].plot([5e-1]+lead_times,backorders,"-.b",alpha=0.2)
    ax[1].plot([5e-1]+lead_times,backorders,"o",label=n_bufs)
    
    ax[2].plot([5e-1]+lead_times,stout_durs,"-.b",alpha=0.2)
    ax[2].plot([5e-1]+lead_times,stout_durs,"o",label=n_bufs)

for i in range(3):
    ax[i].set_xscale("log")
    ax[i].set_xlabel("Supply lead time (days)")
    ax[i].grid(axis="y")
    ax[i].legend(title="Number of intermediate buffers")

ax[0].set_yticks(np.arange(0.9,0.97,0.01))
ax[0].set_ylabel("Effective SLA")
ax[1].set_ylabel("Mean backorders queue (items)")
ax[2].set_ylabel("Mean stockout duration (days)")
ax[0].set_title("Effect of cascading stockouts in presence of intermediate buffer stages")
ax[1].set_yscale("log")
ax[2].set_yscale("log")

In [ ]:
stages

In [ ]:
seed = 8675310
f,ax = plt.subplots(3,1,figsize=(12,18),sharex=False)

stages = create_serial_line_from_lead_times(lead_times=[1],demand_stage_params=demand_stage_params)
policy = {stage_id:{"s":0,"si":0} for stage_id in stages}
base_stocks = compute_base_stocks(stages,policy,lam,sla=sla)

stat_func = np.mean
    
np.random.seed(seed=seed)
inv_histories = simulate(stages,policy,base_stocks,lam,n,stockout_stages=None)
indep_inv_history = inv_histories["1"]
indep_sla = np.mean(inv_histories["1"] >= 0)
indep_stockout_intervals = _collect_stockout_intervals(inv_histories["1"])
indep_backorders = stat_func(sum(indep_stockout_intervals.values(),[]))
indep_stout_dur = stat_func([len(interval) for interval in indep_stockout_intervals.values()])


n_bufs = 4
i = 0
for service_times in enumerate_serial_service_times():
    i+=1
    if i >5:
        break
    service_times = service_times[::-1]+[0]
    effective_slas = [indep_sla]
    backorders = [indep_backorders]
    stout_durs = [indep_stout_dur]
    effective_slas_2 = [indep_sla]
    for l2 in lead_times:
        l_times = [1]+[1]*n_bufs+[l2]
        stages = create_serial_line_from_lead_times(lead_times=l_times,demand_stage_params=demand_stage_params)
        policy = {stage_id:{"s":service_times[int(stage_id)-1],"si":service_times[int(stage_id)]} for stage_id in stages}
        #print(policy)
        rep_times = compute_replenishment_times(policy,stages)
        #print(rep_times)
        base_stocks = compute_base_stocks(stages,policy,lam,sla=sla)

        np.random.seed(seed=seed)
        inv_histories = simulate(stages,policy,base_stocks,lam,n,stockout_stages=None)
        effective_slas.append(np.mean(inv_histories["1"][l2:] >= 0))
        stockout_intervals = get_new_stockout_intervals(inv_histories["1"],indep_inv_history)
        
        backorders.append(stat_func(sum(stockout_intervals.values(),[])))
        stout_durs.append(stat_func([len(inter) for inter in stockout_intervals.values()]))
        
        #neg = inv_histories["1"][l2:] < 0
        #bad_days = ~(neg & indep_neg[l2:]) & neg
        #backorders.append(-np.median(inv_histories["1"][l2:][bad_days]))

    ax[0].plot([5e-1]+lead_times,effective_slas,"-.b",alpha=0.2)
    ax[0].plot([5e-1]+lead_times,effective_slas,"o",label="{}".format(rep_times))
    
    ax[1].plot([5e-1]+lead_times,backorders,"-.b",alpha=0.2)
    ax[1].plot([5e-1]+lead_times,backorders,"o",label="{}".format(rep_times))
    
    ax[2].plot([5e-1]+lead_times,stout_durs,"-.b",alpha=0.2)
    ax[2].plot([5e-1]+lead_times,stout_durs,"o",label="{}".format(rep_times))

for i in range(3):
    ax[i].set_xscale("log")
    ax[i].set_xlabel("Supply lead time (days)")
    ax[i].grid(axis="y")
    ax[i].legend(title="Number of intermediate buffers")

ax[0].set_yticks(np.arange(0.9,0.97,0.01))
ax[0].set_ylabel("Effective SLA")
ax[1].set_ylabel("Mean backorders queue (items)")
ax[2].set_ylabel("Mean stockout duration (days)")
ax[0].set_title("Effect of cascading stockouts in presence of intermediate buffer stages")
ax[1].set_yscale("log")
ax[2].set_yscale("log")

In [ ]:
def enumerate_serial_service_times(service_times_list = [0]):
    if len(service_times_list) == 5:
        service_times_list.append(0)
        yield service_times_list
        return

    for s in range(service_times_list[-1]+1+1):
        new_service_times_list = service_times_list + [s]
        for full_list in enumerate_serial_service_times(new_service_times_list):
            yield full_list

In [ ]:
for service_times in enumerate_serial_service_times():
    service_times = service_times[::-1]
    print(service_times)

In [ ]:
plt.figure(figsize=(12,8))
stockout_stages = []
effective_slas = []
for stage_id in range(1,len(stages)+1):
    stockout_stages.append(str(stage_id))
    np.random.seed(seed=8675309)
    inv_histories = simulate(stages,policy,base_stocks,lam,n,stockout_stages=stockout_stages)
    effective_slas.append(np.mean(inv_histories["1"] >= 0))
    print(np.min(inv_histories["1"]))
plt.plot(effective_slas,"b",alpha=0.5)
plt.plot(range(0,len(stages)),effective_slas,"o")

In [ ]:
plt.figure(figsize=(12,8))
stockout_stages = []
effective_slas = []
for stage_id in range(1,len(stages)+1):
    stockout_stages.append(str(stage_id))
    np.random.seed(seed=8675309)
    inv_histories = simulate(stages,policy,base_stocks,lam,n,stockout_stages=stockout_stages)
    effective_slas.append(np.mean(inv_histories["1"] >= 0))
    print(np.min(inv_histories["1"]))
plt.plot(effective_slas,"b",alpha=0.5)
plt.plot(range(0,len(stages)),effective_slas,"o")

In [ ]:
plt.figure(figsize=(12,8))
for profiles_dict in iterate_experimental_profiles():
    stages = create_serial_stages(**profiles_dict)
    stages["1"].demand_mean = lam
    stages["1"].demand_std = np.sqrt(lam)
    solution = GuaranteedServiceModelTree(stages).find_optimal_solution()
    policy = solution.policy
    base_stocks = solution.base_stocks
    
    #policy = {stage_id:{"s":0,"si":0} for stage_id in stages}
    #base_stocks = compute_base_stocks(stages,policy,lam,sla)

    stockout_stages = []
    effective_slas = []
    for stage_id in range(1,6):
        stockout_stages.append(str(stage_id))
        np.random.seed(seed=8675309)
        inv_histories = simulate(stages,policy,base_stocks,lam,n,stockout_stages=stockout_stages)
        effective_slas.append(np.mean(inv_histories["1"] >= 0))
    plt.plot(effective_slas,"b",alpha=0.5)
    plt.plot(range(0,5),effective_slas,"o",label=profiles_dict)

plt.xticks(range(0,5))
plt.grid(axis="y")
plt.xlabel("Number of stages with propagating stockouts",fontsize=15)
plt.ylabel("Effective SLA",fontsize=15)

plt.legend()

In [ ]:
base_stocks

In [ ]:
np.mean(inv_histories["3"] < 0 )

In [ ]:
for profiles_dict in iterate_experimental_profiles():
    plt.figure(figsize=(12,8))
    stages = create_serial_stages(**profiles_dict)
    stages["1"].demand_mean = lam
    stages["1"].demand_std = np.sqrt(lam)
    solution = GuaranteedServiceModelTree(stages).find_optimal_solution()
    policy = solution.policy
    base_stocks = solution.base_stocks

    stockout_stages = []
    effective_slas = []
    for stage_id in range(1,6):
        stockout_stages.append(str(stage_id))
        np.random.seed(seed=8675309)
        inv_histories = simulate(stages,policy,base_stocks,lam,n,stockout_stages=stockout_stages)
        effective_slas.append(np.mean(inv_histories["1"] >= 0))
    plt.plot(effective_slas,"b",alpha=0.5)
    plt.plot(range(0,5),effective_slas,"bo",label=profiles_dict)
    
    policy = {stage_id:{"s":0,"si":0} for stage_id in stages}
    base_stocks = compute_base_stocks(stages,policy,lam,sla)
    
    stockout_stages = []
    effective_slas = []
    for stage_id in range(1,6):
        stockout_stages.append(str(stage_id))
        np.random.seed(seed=8675309)
        inv_histories = simulate(stages,policy,base_stocks,lam,n,stockout_stages=stockout_stages)
        effective_slas.append(np.mean(inv_histories["1"] >= 0))
    plt.plot(effective_slas,"r",alpha=0.5)
    plt.plot(range(0,5),effective_slas,"ro",label=profiles_dict)
    
    
    plt.xticks(range(0,5))
    plt.grid(axis="y")
    plt.xlabel("Number of stages with propagating stockouts",fontsize=15)
    plt.ylabel("Effective SLA",fontsize=15)

    plt.legend()

In [ ]:
poisson

In [ ]:
stages = read_supply_chain_from_txt("snc/experiment/basic_serial_network_config.txt")
policy = {"Demand":{"s":0,"si":3},"Dist":{"s":3,"si":0}}

In [ ]:
stages["Demand"].lead_time = 1
stages["Dist"].lead_time = 39

In [ ]:
stages = read_supply_chain_from_txt("snc/experiment/basic_serial_network.txt")
policy = {"Dist":{"s":3,"si":0},"Demand":{"s":0,"si":3},"Supply":{"s":0,"si":0}}

In [ ]:
stages = read_supply_chain_from_txt("snc/experiment/basic_serial_network.txt")
policy = {"Dist":{"s":3,"si":0},"Demand":{"s":0,"si":3},"Supply":{"s":0,"si":0}}

In [ ]:
n=1000
sla = 0.95
p_bf = 0.01
lam = 10
bf = 100

np.random.seed(seed=8675309)
demand_history = np.random.poisson(size=n,lam=lam)
#n_bf = np.random.binomial(n,p_bf)

#idx_bf = np.random.choice(np.arange(len(demand_history)),n_bf)
#demand_history[idx_bf] = bf

In [ ]:
demand_history.mean()

In [ ]:
demand_history.var()

In [ ]:
(12-demand_history.mean())/demand_history.std()

In [ ]:
base_stocks = compute_base_stocks(stages,policy,lam,sla)
#base_stocks["Dist"] = 864
#base_stocks["Demand"] = 106
#base_stocks["Dist"] = 885
#print(582/base_stocks["Dist"])
print(585/base_stocks["Dist"])
print(110/base_stocks["Demand"])

base_stocks["Dist"] = 393
#base_stocks["Demand"] = 110

#base_stocks["Dist"] = 401

capacity_constraints = {}
#capacity_constraints = {"Demand":12}

indep_inv_histories = simulate(stages,policy,base_stocks,capacity_constraints,demand_history,stockout_stages=[])

casc_inv_histories = simulate(stages,policy,base_stocks,capacity_constraints,demand_history,stockout_stages=None)

#capacity_constraints = {"Demand":13,"Dist":12}
capacity_constraints = {"Demand":35,"Dist":31}
capacity_constraints = {"Dist":12}

indep_inv_histories_cap = simulate(stages,policy,base_stocks,capacity_constraints,demand_history,stockout_stages=[])

casc_inv_histories_cap = simulate(stages,policy,base_stocks,capacity_constraints,demand_history,stockout_stages=None)

In [ ]:
base_stocks

In [ ]:
#now check the effective sla with coupled stockouts
np.mean(casc_inv_histories_cap["Demand"] >= 0),np.mean(casc_inv_histories_cap["Dist"] >= 0)

In [ ]:
#verify stockout frequency against sla
np.mean(indep_inv_histories["Demand"] >= 0),np.mean(indep_inv_histories["Dist"] >= 0)

In [ ]:
#verify stockout frequency against sla
np.mean(indep_inv_histories_cap["Demand"] >= 0),np.mean(indep_inv_histories_cap["Dist"] >= 0)

In [ ]:
#check the correlation between inventories
np.corrcoef(indep_inv_histories["Demand"][100:n],indep_inv_histories["Dist"][100:n])

In [ ]:
#now check the effective sla with coupled stockouts
np.mean(casc_inv_histories["Demand"] >= 0),np.mean(casc_inv_histories["Dist"] >= 0)

In [ ]:
#now check the effective sla with coupled stockouts
np.mean(casc_inv_histories_cap["Demand"] >= 0),np.mean(casc_inv_histories_cap["Dist"] >= 0)

In [ ]:
#and corresponding correlation between inventories
np.corrcoef(indep_inv_histories_cap["Dist"][100:n],indep_inv_histories["Dist"][100:n])

In [ ]:
len(demand_history.shape)

In [ ]:
plt.plot(demand_history[:200])

In [ ]:
plot_cascading_effect(casc_inv_histories_cap,casc_inv_histories,["Dist","Demand"],time_length=n,remove_transient=True)

In [ ]:
plot_cascading_effect(indep_inv_histories_cap,indep_inv_histories,["Dist","Demand"],time_length=n,remove_transient=True)

In [ ]:
#find one coupled stockout and plot it
loc = np.where(casc_inv_histories_cap["Demand"]<-200)[0][0]
print(loc)

window = 2000
s = loc-window
e = s+2*window
plt.figure(figsize=(12,8))
for stage_id in casc_inv_histories:
    plt.plot(casc_inv_histories_cap[stage_id][s:e],label="{} stage inventory position".format(stage_id))
#plt.plot(casc_inv_histories["Demand"][s:e],label="Demand stage inventory position")
#plt.plot(casc_inv_histories["Dist"][s:e],label="Supply stage inventory position")
#plt.plot(casc_inv_histories["Supply"][s:e],label="Supply stage inventory position")
plt.ylabel("Inventory position")
plt.xlabel("Day")
plt.grid(axis="y")
plt.legend()

In [ ]:
window = 100
s = loc-window
e = s+2*window
plt.figure(figsize=(12,8))
#plt.plot(indep_inv_histories["Demand"][s:e],label="{}: no capacity constraint".format(stage_id))
#plt.plot(indep_inv_histories_cap["Demand"][s:e],label="{}: capacity constraint".format(stage_id))
plt.plot(casc_inv_histories_cap["Demand"][s:e],label="{}: capacity constraint, cascade".format(stage_id))
plt.plot(casc_inv_histories["Demand"][s:e],label="{}: no capacity constraint, cascade".format(stage_id))
plt.plot(indep_inv_histories_cap["Demand"][s:e],label="{}: no cascade".format(stage_id))

plt.ylabel("Inventory position")
plt.xlabel("Day")
plt.grid(axis="y")
plt.legend()

In [ ]:
from collections import defaultdict

In [ ]:
new_stockouts = get_new_stockout_intervals(casc_inv_histories["Demand"],indep_inv_histories["Demand"])
new_stockouts_cap = get_new_stockout_intervals(casc_inv_histories_cap["Demand"],indep_inv_histories_cap["Demand"])

In [ ]:
indep_stockouts_cap = _collect_stockout_intervals(indep_inv_histories_cap["Demand"])
indep_stockouts = _collect_stockout_intervals(indep_inv_histories["Demand"])

In [ ]:
all_stockouts_cap = _collect_stockout_intervals(casc_inv_histories_cap["Demand"])
all_stockouts = _collect_stockout_intervals(casc_inv_histories["Demand"])

In [ ]:
sum([len(inter) for inter in all_stockouts_cap.values()])/sum([len(inter) for inter in indep_stockouts_cap.values()])

In [ ]:
sum([len(inter) for inter in all_stockouts.values()])/sum([len(inter) for inter in indep_stockouts.values()])

In [ ]:
_collect_stockout_intervals(casc_inv_histories_cap["Demand"])

In [ ]:
len(new_stockouts_cap)/len(_collect_stockout_intervals(casc_inv_histories_cap["Demand"]))

In [ ]:
np.max([len(inter) for inter in new_stockouts.values()])

In [ ]:
np.max([len(inter) for inter in new_stockouts_cap.values()])

In [ ]:
from collections import Counter

In [ ]:
n/365

In [ ]:
Counter([len(inter) for inter in new_stockouts_cap.values()])

In [ ]:
for i,j in new_stockouts_cap.items():
    if len(j) == 61:
        print(i)

In [ ]:
f,ax = plt.subplots(2,1,sharex=True)
ax[1].hist([len(inter) for inter in new_stockouts_cap.values()],density=True)
ax[0].hist([len(inter) for inter in new_stockouts.values()],density=True)

In [ ]:
30/32

In [ ]:
window = 50
s = loc-window
e = s+2*window
plt.figure(figsize=(12,8))
plt.plot(indep_inv_histories["Demand"][s:e],label="{}: no capacity constraint".format(stage_id))
plt.plot(indep_inv_histories_cap["Demand"][s:e],label="{}: capacity constraint".format(stage_id))
plt.plot(casc_inv_histories["Demand"][s:e],label="{}: no capacity constraint, cascade".format(stage_id))
plt.plot(casc_inv_histories_cap["Demand"][s:e],label="{}: capacity constraint, cascade".format(stage_id))

plt.ylabel("Inventory position")
plt.xlabel("Day")
plt.grid(axis="y")
plt.legend()

In [ ]:
#find one coupled stockout and plot it
loc = np.where(casc_inv_histories_cap["Demand"]< -30)[0][5]

window = 150
s = loc-window
e = s+2*window
f,ax = plt.subplots(4,1,figsize=(12,12),sharex=True)

for j,inv_hist in enumerate([indep_inv_histories,indep_inv_histories_cap,casc_inv_histories,casc_inv_histories_cap]):

    for stage_id in inv_hist:
        ax[j].plot(inv_hist[stage_id][s:e],label="{} stage inventory position".format(stage_id))
    
    ax[j].grid(axis="y")
#plt.plot(casc_inv_histories["Demand"][s:e],label="Demand stage inventory position")
#plt.plot(casc_inv_histories["Dist"][s:e],label="Supply stage inventory position")
#plt.plot(casc_inv_histories["Supply"][s:e],label="Supply stage inventory position")
plt.ylabel("Inventory position")
plt.xlabel("Day")
plt.legend()

In [ ]:
window = 150
s = loc-window
e = s+2*window
plt.figure(figsize=(12,8))
for stage_id in casc_inv_histories:
    plt.plot(indep_inv_histories_cap[stage_id][s:e],label="{} stage inventory position".format(stage_id))
#plt.plot(casc_inv_histories["Demand"][s:e],label="Demand stage inventory position")
#plt.plot(casc_inv_histories["Dist"][s:e],label="Supply stage inventory position")
#plt.plot(casc_inv_histories["Supply"][s:e],label="Supply stage inventory position")
plt.ylabel("Inventory position")
plt.xlabel("Day")
plt.grid(axis="y")
plt.legend()

In [ ]:
window = 150
s = loc-window
e = s+2*window
plt.figure(figsize=(12,8))
for stage_id in casc_inv_histories:
    plt.plot(casc_inv_histories[stage_id][s:e],label="{} stage inventory position".format(stage_id))
#plt.plot(casc_inv_histories["Demand"][s:e],label="Demand stage inventory position")
#plt.plot(casc_inv_histories["Dist"][s:e],label="Supply stage inventory position")
#plt.plot(casc_inv_histories["Supply"][s:e],label="Supply stage inventory position")
plt.ylabel("Inventory position")
plt.xlabel("Day")
plt.grid(axis="y")
plt.legend()

In [ ]:
window = 150
s = loc-window
e = s+2*window
plt.figure(figsize=(12,8))
for stage_id in casc_inv_histories:
    plt.plot(indep_inv_histories[stage_id][s:e],label="{} stage inventory position".format(stage_id))
#plt.plot(casc_inv_histories["Demand"][s:e],label="Demand stage inventory position")
#plt.plot(casc_inv_histories["Dist"][s:e],label="Supply stage inventory position")
#plt.plot(casc_inv_histories["Supply"][s:e],label="Supply stage inventory position")
plt.ylabel("Inventory position")
plt.xlabel("Day")
plt.grid(axis="y")
plt.legend()

In [ ]:
np.random.seed(seed=8675309)
np.random.poisson(size=20,lam=10)

In [ ]:
plt.plot(indep_inv_histories["Demand"])

In [ ]:
#compare it with the same time interval without the coupled inventory dynamics
window = 200
s = loc-window
e = s+2*window
plt.figure(figsize=(12,8))
plt.plot(indep_inv_histories["Dist"][s:e],label="Demand stage inventory position")
plt.plot(indep_inv_histories_cap["Dist"][s:e],label="Supply stage inventory position")
plt.ylabel("Inventory position")
plt.xlabel("Day")
plt.grid(axis="y")
plt.legend()